In [5]:
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# ======== 1. Cargar todos los archivos CSV ========
archivos = sorted(glob.glob("Objeto_*_ES_pos_num.csv"))
dataframes = []

for f in archivos:
    df = pd.read_csv(f)
    df["vehiculo"] = f.split("_")[1]  # extraer el número del nombre
    dataframes.append(df)

data = pd.concat(dataframes, ignore_index=True)

2025-10-04 00:54:17.180140: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Asegurar el orden temporal
data = data.sort_values(by=["vehiculo", "Año", "Mes", "Día", "Hora", "Minuto", "Segundo"])

# Seleccionamos variables relevantes
features = ["Velocidad (km/h)"]
target = "is_microparada"

# Normalizar velocidad (importante para redes recurrentes)
scaler = MinMaxScaler()
data["Velocidad_norm"] = scaler.fit_transform(data[["Velocidad (km/h)"]])

In [20]:
def crear_ventanas(df, n_pasos=32):
    X, y = [], []
    for i in range(len(df) - n_pasos):
        X.append(df.iloc[i:i+n_pasos][["Velocidad_norm"]].values)
        y.append(df.iloc[i+n_pasos][target])
    return np.array(X), np.array(y)

X_total, y_total = [], []

for idv, grupo in data.groupby("vehiculo"):
    X, y = crear_ventanas(grupo.reset_index(drop=True), n_pasos=20)
    X_total.append(X)
    y_total.append(y)

X = np.concatenate(X_total)
y = np.concatenate(y_total)

print("Forma de X:", X.shape)  # (muestras, 20, 1)
print("Forma de y:", y.shape)

# ======== 4. División en entrenamiento y validación ========
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

Forma de X: (1994, 20, 1)
Forma de y: (1994,)


In [8]:
model = Sequential([
    GRU(64, input_shape=(20, 1), return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # salida binaria
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

W0000 00:00:1759557316.739793  146698 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/antonchigurh/global-env/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        12,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,977 (58.50 KB)

 Trainable params: 14,977 (58.50 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=80,
    batch_size=16,
    verbose=1
)

Epoch 1/80
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7285 - loss: 0.5416 - val_accuracy: 0.7168 - val_loss: 0.5568
Epoch 2/80
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7323 - loss: 0.5407 - val_accuracy: 0.7168 - val_loss: 0.5565
Epoch 3/80
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7329 - loss: 0.5386 - val_accuracy: 0.7218 - val_loss: 0.5545
Epoch 4/80
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7348 - loss: 0.5389 - val_accuracy: 0.7318 - val_loss: 0.5553
Epoch 5/80
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7310 - loss: 0.5425 - val_accuracy: 0.7218 - val_loss: 0.5558
Epoch 6/80
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7310 - loss: 0.5398 - val_accuracy: 0.7343 - val_loss: 0.5570
Epoch 7/80
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7379 - loss: 0.5400 - val_accuracy: 0.7243 - val_loss: 0.5544
Epoch 8/80
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7342 - loss: 0.5405 - val_accurac

In [22]:
y_pred = (model.predict(X_val) > 0.5).astype(int)

print("\nMatriz de confusión:")
print(confusion_matrix(y_val, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_val, y_pred))

# ======== 8. Guardar modelo ========
model.save("modelo_microparadas_gru.h5")
print("\nModelo guardado en: modelo_microparadas_gru.h5")

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 



Matriz de confusión:
[[213  50]
 [ 70  66]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.75      0.81      0.78       263
           1       0.57      0.49      0.52       136

    accuracy                           0.70       399
   macro avg       0.66      0.65      0.65       399
weighted avg       0.69      0.70      0.69       399


Modelo guardado en: modelo_microparadas_gru.h5


In [19]:
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# ======== 1. Cargar todos los archivos CSV ========
archivos = sorted(glob.glob("Objeto_*_ES_pos_num.csv"))
dataframes = []

for f in archivos:
    df = pd.read_csv(f)
    df["vehiculo"] = f.split("_")[1]
    dataframes.append(df)

data = pd.concat(dataframes, ignore_index=True)

# ======== 2. Preprocesamiento ========
data = data.sort_values(by=["vehiculo", "Año", "Mes", "Día", "Hora", "Minuto", "Segundo"])

features = ["Velocidad (km/h)"]
target = "is_microparada"

scaler = MinMaxScaler()
data["Velocidad_norm"] = scaler.fit_transform(data[["Velocidad (km/h)"]])

# ======== 3. Crear secuencias (ventanas de 20 lecturas) ========
def crear_ventanas(df, n_pasos=40):
    X, y = [], []
    for i in range(len(df) - n_pasos):
        X.append(df.iloc[i:i+n_pasos][["Velocidad_norm"]].values)
        y.append(df.iloc[i+n_pasos][target])
    return np.array(X), np.array(y)

X_total, y_total = [], []

for idv, grupo in data.groupby("vehiculo"):
    X, y = crear_ventanas(grupo.reset_index(drop=True), n_pasos=20)
    X_total.append(X)
    y_total.append(y)

X = np.concatenate(X_total)
y = np.concatenate(y_total)

print("Forma de X:", X.shape)
print("Forma de y:", y.shape)

# ======== 4. División en entrenamiento y validación ========
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ======== 5. Modelo LSTM ========
model = Sequential([
    LSTM(64, input_shape=(20, 1), return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# ======== 6. Entrenamiento ========
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=40,
    batch_size=16,
    verbose=1
)

# ======== 7. Evaluación ========
y_pred = (model.predict(X_val) > 0.5).astype(int)

print("\nMatriz de confusión:")
print(confusion_matrix(y_val, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_val, y_pred))

# ======== 8. Guardar modelo ========
model.save("modelo_microparadas_lstm.h5")
print("\nModelo guardado en: modelo_microparadas_lstm.h5")


Forma de X: (1994, 20, 1)
Forma de y: (1994,)


/home/antonchigurh/global-env/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,009 (74.25 KB)

 Trainable params: 19,009 (74.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.6552 - loss: 0.6359 - val_accuracy: 0.6591 - val_loss: 0.6250
Epoch 2/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6602 - loss: 0.6125 - val_accuracy: 0.6591 - val_loss: 0.6133
Epoch 3/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6727 - loss: 0.6019 - val_accuracy: 0.6867 - val_loss: 0.6044
Epoch 4/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7003 - loss: 0.5902 - val_accuracy: 0.6992 - val_loss: 0.5920
Epoch 5/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7204 - loss: 0.5801 - val_accuracy: 0.7218 - val_loss: 0.5809
Epoch 6/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.7166 - loss: 0.5705 - val_accuracy: 0.7268 - val_loss: 0.5740
Epoch 7/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7229 - loss: 0.5640 - val_accuracy: 0.7143 - val_loss: 0.5776
Epoch 8/40
100/100 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.7266 - loss: 0.5591 - val_accu


Matriz de confusión:
[[226  37]
 [ 73  63]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.76      0.86      0.80       263
           1       0.63      0.46      0.53       136

    accuracy                           0.72       399
   macro avg       0.69      0.66      0.67       399
weighted avg       0.71      0.72      0.71       399


Modelo guardado en: modelo_microparadas_lstm.h5
